In [ ]:
#Pinecone Upload  - PREPORCEESING DATA SAVER

In [3]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.


In [4]:
from google.cloud import storage
import json
import pandas as pd
from pinecone import Pinecone
import time
from google import genai
from google.genai import types
import base64
import time
from google.api_core import exceptions as g_exceptions

In [5]:
# Initialize GCS client
client = storage.Client()
bucket_name = "finetune-data123"
bucket = client.bucket(bucket_name)

In [20]:
filepath_court = "main_pipiline_data/court"
filepath_tribunals = "main_pipiline_data/tribunals/tribunals"

def get_gcs_file_paths(bucket, court="", year=""):
    paths = []

    # Iterate over all blobs in the bucket
    for blob in bucket.list_blobs():  # optionally use prefix to speed up
        if not blob.name.endswith('.xml'):
            continue
        # Apply filters
        if court and court not in blob.name:
            continue
        if year and year not in blob.name:
            continue
        paths.append(blob.name)

    return paths

all_files = get_gcs_file_paths(bucket)

In [22]:
import re
import json
from bs4 import BeautifulSoup
from google.cloud import storage

class FileProcessor:
    def __init__(self, bucket_name):
        self.client = storage.Client()
        self.bucket = self.client.bucket(bucket_name)

    def clean_text(self, text: str) -> str:
        text = re.sub(r'\n{2,}', '\n', text)
        text = re.sub(r' +', ' ', text)
        text = text.strip()
        # remove any line that begins with #judgment
        text = re.sub(r'(?m)^#judgment.*\n?', '', text)
        return text

    def get_metadata(self, soup, path):
      metadata = {"path": path}

      # FRBRWork metadata
      frbrwork = soup.find('frbrwork')
      if frbrwork:
          frbr_tags = {
              'frbruri': 'uri',
              'frbrdate': 'judgment_date',
              'frbrname': 'name',
              'frbrauthor': 'author',
              'frbrcountry': 'country',
              'frbrnumber': 'number',
              'frbrthis': 'this'
          }
          for tag_name, key in frbr_tags.items():
              tag = frbrwork.find(tag_name)
              if tag:
                  # Some have 'value', some have 'date', some have 'href'
                  value = tag.attrs.get('value') or tag.attrs.get('date') or tag.attrs.get('href')
                  if value:
                      metadata[key] = value

      # Capture proprietary UK metadata (namespaced)
      uk_tags = ['uk:court', 'uk:year', 'uk:number', 'uk:cite']
      for tag_name in uk_tags:
          tag = soup.find(tag_name)
          if tag:
              metadata[tag_name.replace('uk:', '')] = tag.get_text(strip=True)

      # Capture judges from references
      judges = [p.get('showas') for p in soup.find_all('tlcperson')]
      if judges:
          metadata['judges'] = judges

      return metadata


    def process_files(self, filepaths):
        """
        filepaths: list of paths in the bucket
        Returns a list of dicts: [{'metadata': ..., 'text': ...}, ...]
        """
        results = []
        i = 0
        for path in filepaths:
            i+=1
            print(f"{i}/{len(filepaths)}")
            blob = self.bucket.blob(path)
            # Download as string
            xml_content = blob.download_as_text()  # memory-efficient for one file at a time
            soup = BeautifulSoup(xml_content, "lxml")  # parse XML
            metadata = self.get_metadata(soup, path)

            # Get main text (adjust selector depending on your XML structure)
            text_tag = soup.find('Text') or soup  # fallback to whole document
            text = text_tag.get_text() if text_tag else ''
            clean = self.clean_text(text)

            results.append({
                'metadata': metadata,
                'text': clean
            })

        return results



In [23]:
bucket_name = "finetune-data123"
filepaths = all_files

processor = FileProcessor(bucket_name)
processed_files = processor.process_files(filepaths)
# save results in a results file ni fietune-dataq

import json
from google.cloud import storage

def save_results_to_gcs(bucket_name, filepath, results):
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    blob = bucket.blob(filepath)

    blob.upload_from_string(
        data=json.dumps(results, indent=2, ensure_ascii=False),
        content_type="application/json"
    )
    print(f"✅ Results saved to gs://{bucket_name}/{filepath}")


# After processing all files
output_filename = "main_pipiline_data/preprocessed/processed_results.json"
save_results_to_gcs(bucket_name, output_filename, processed_files)


2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100
43/100
44/100
45/100
46/100
47/100
48/100
49/100
50/100
51/100
52/100
53/100
54/100
55/100
56/100
57/100
58/100
59/100
60/100
61/100
62/100
63/100
64/100
65/100
66/100
67/100
68/100
69/100
70/100
71/100
72/100
73/100
74/100
75/100
76/100
77/100
78/100
79/100
80/100
81/100
82/100
83/100
84/100
85/100
86/100
87/100
88/100
89/100
90/100
91/100
92/100
93/100
94/100
95/100
96/100
97/100
98/100
99/100
100/100
101/100
✅ Results saved to gs://finetune-data123/main_pipiline_data/preprocessed/processed_results.json
